In [ ]:
# This note was created to check duplicate, missing value, outliers, dublication and summarize and visulize opendata:public_odl_brutto_1h
from owslib.wfs import WebFeatureService
import json
from owslib.fes import *

# get WFS
wfs11 = WebFeatureService(url='https://entw-imis.lab.bfs.de/ogc/opendata/wfs', version='1.1.0')

# apply filtes
filter1 = PropertyIsBetween(propertyname='start', lower='2021-01-01' ,upper= '2022-12-31')
filter2 = PropertyIsLike(propertyname='id', literal='DEZ3159',wildCard='*')
filters=[filter1,filter2]

# get respose
filterxml = etree.tostring(And(operations=filters).toXML()).decode("utf-8")
response = wfs11.getfeature(typename='opendata:public_odl_brutto_1h',filter = filterxml,outputFormat='application/json')

# convert IO-byte to bytes
bytesD=bytes(response.read())
# convert to json
data2 = json.loads(bytesD)


In [ ]:
#save json to show in qgis
import json
with open('Halberstadt' + '.json', 'w', encoding='utf-8') as f:
    json.dump(data2, f, ensure_ascii=False, indent=4)

In [ ]:
# convert to data layer
import pandas as pd
datanorm1= pd.json_normalize(data2,"features")
df2=pd.DataFrame(datanorm1)



In [ ]:
#Describe data
print(df2.sum())
print(df2.info())
print(df2["properties.value"].describe())


In [ ]:
# check duplicate
duplicatef2 = df2[df2['properties.end_measure'].duplicated()]
print(duplicatef2)

In [ ]:
#prepare data to plot
from IPython.display import display
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


df2['properties.end_measure'] = pd.to_datetime(df2['properties.end_measure']).dt.tz_localize(None)
df2['properties.start'] = pd.to_datetime(df2['properties.start']).dt.tz_localize(None)
df2 = df2.set_index(df2['properties.end_measure'])


In [ ]:
# dates which are not in the sequence are returned
missv=pd.date_range(start="2021-01-01 00:00:00", end="2022-12-31 23:00:00", freq = 'H').difference(df2.index)
#print(missv.sort_values(ascending=True))
print(pd.DataFrame(missv).count())
missv = np.array(pd.to_datetime(missv.sort_values(ascending=True)))
print(missv)

In [ ]:
# resample daily
import seaborn as sns

df2=df2.resample('D').mean()


In [ ]:
# check null values
isnulldf2 = df2[df2['properties.value'].isnull()]
print(isnulldf2)

In [ ]:

# time series plot
import matplotlib.dates as mdates
fig, ax = plt.subplots(figsize=(11, 4))
year_month_formatter = mdates.DateFormatter("%Y-%m") # four digits for year, two for month
ax.xaxis.set_major_formatter(year_month_formatter) # formatter for major axis only
ax.plot(df2.index.values,df2['properties.value'],linewidth=0.7,color='red')

plt.xlabel('Date')
plt.ylabel('AGDR (μSv/d)')

In [ ]:
# Scatter Plot
fig, ax = plt.subplots(figsize=(11, 4))

# Add x-axis and y-axis
ax.scatter(df2.index.values,
           df2['properties.value'],
           color='red')
ax.set(xlabel="Date",
       ylabel="AGDR (μSv/d)")

plt.show()

In [ ]:
df2=df2.resample('W').mean()
sns.set(rc={'figure.figsize':(11, 4)})
df2['properties.value'].plot(linewidth=0.9)

In [ ]:

df2=df2.resample('W').mean()
# create the time series plot
sns.set_style()
  
sns.lineplot(x = df2.index.values, y = "properties.value",
             data = df2)
  
#plt.xticks(rotation = 25)

In [ ]:
from pandas import DataFrame
from pandas import Grouper
from pandas import concat
import seaborn as sns

df2 = df2.resample('M').mean()
df2 = df2.sort_index().loc['2021-01-01':'2021-12-31']
print(df2.head())

sns.set_style("whitegrid")
  
sns.boxplot(x = df2.index.values, y = df2['properties.value'], data = df2)

In [1]:
import seaborn as sns
df1=df2.resample('D').sum()
sns.set(rc={'figure.figsize':(11, 4)})
# Create figure and plot space
fig, ax = plt.subplots(figsize=(8, 5))

# Add x-axis and y-axis
ax.scatter(df2.index.values,
           df2['properties.value'],
           color='blue')

# Set title and labels for axes
ax.set(xlabel="Date",
       ylabel="ODL (μSv/h)")
df2['properties.value'].plot(linewidth=0.5)

NameError: name 'df2' is not defined